In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from collections import Counter
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import re
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import StratifiedKFold

# -----------------------------
# 1. Text Preprocessing Classes
# -----------------------------
class TextPreprocessor:
    def __init__(self, max_vocab_size=15000, max_seq_length=128):
        self.max_vocab_size = max_vocab_size
        self.max_seq_length = max_seq_length
        self.word2idx = {'<PAD>': 0, '<UNK>': 1}
        self.idx2word = {0: '<PAD>', 1: '<UNK>'}
        self.word_counts = Counter()
        
    def clean_text(self, text):
        text = str(text).lower()
        text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)
        text = ' '.join(text.split())
        return text
        
    def fit(self, texts):
        for text in texts:
            cleaned_text = self.clean_text(text)
            words = cleaned_text.split()
            self.word_counts.update(words)
        
        # Reserve two indices for PAD and UNK tokens
        vocab_words = [word for word, count in self.word_counts.most_common(self.max_vocab_size - 2)]
        for word in vocab_words:
            idx = len(self.word2idx)
            self.word2idx[word] = idx
            self.idx2word[idx] = word
    
    def transform(self, texts):
        sequences = []
        for text in texts:
            cleaned_text = self.clean_text(text)
            words = cleaned_text.split()
            # Truncate or pad sequences
            seq = [self.word2idx.get(word, self.word2idx['<UNK>']) for word in words[:self.max_seq_length]]
            seq = seq + [self.word2idx['<PAD>']] * (self.max_seq_length - len(seq))
            sequences.append(seq)
        return torch.tensor(sequences)

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# -----------------------------
# 2. Define the Model
# -----------------------------
class ImprovedTextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_layers=2):
        super(ImprovedTextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.embedding_dropout = nn.Dropout(0.2 if num_layers > 1 else 0.1)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, 
                            batch_first=True, bidirectional=True,
                            dropout=0.2 if num_layers > 1 else 0)
        self.attention = nn.Linear(hidden_dim * 2, 1)
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.bn2 = nn.BatchNorm1d(hidden_dim // 2)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(hidden_dim // 2, num_classes)
        
    def attention_net(self, lstm_output):
        # Compute attention weights and context vector
        energy = torch.tanh(self.attention(lstm_output))
        attention_weights = torch.softmax(self.attention(lstm_output), dim=1)
        context_vector = torch.sum(attention_weights * lstm_output, dim=1)
        return context_vector
        
    def forward(self, x):
        embedded = self.embedding(x)
        embedded = self.embedding_dropout(embedded)
        lstm_out, _ = self.lstm(embedded)
        attn_out = self.attention_net(lstm_out)
        x = self.fc1(attn_out)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# -----------------------------
# 3. Function to Load Pretrained Embeddings (GloVe)
# -----------------------------
def load_pretrained_embeddings(embedding_path, word2idx, embed_dim):
    print("Loading pretrained embeddings...")
    # Initialize embeddings with a uniform distribution
    embeddings = np.random.uniform(-0.05, 0.05, (len(word2idx), embed_dim))
    embeddings[word2idx['<PAD>']] = np.zeros(embed_dim)
    found = 0
    with open(embedding_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.rstrip().split(' ')
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            if vector.shape[0] != embed_dim:
                continue  # Skip if dimensions mismatch
            if word in word2idx:
                embeddings[word2idx[word]] = vector
                found += 1
    print(f"Found {found} pretrained embeddings out of {len(word2idx)} words")
    return torch.tensor(embeddings, dtype=torch.float)

# -----------------------------
# 4. Training Function
# -----------------------------
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, preprocessor, config):
    best_f1 = 0
    patience = 5
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        for batch_texts, batch_labels in train_loader:
            batch_texts, batch_labels = batch_texts.to(device), batch_labels.to(device)
            optimizer.zero_grad()
            outputs = model(batch_texts)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
            predictions = torch.argmax(outputs, dim=1)
            train_preds.extend(predictions.cpu().numpy())
            train_labels.extend(batch_labels.cpu().numpy())
        
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for batch_texts, batch_labels in val_loader:
                batch_texts, batch_labels = batch_texts.to(device), batch_labels.to(device)
                outputs = model(batch_texts)
                loss = criterion(outputs, batch_labels)
                val_loss += loss.item()
                predictions = torch.argmax(outputs, dim=1)
                val_preds.extend(predictions.cpu().numpy())
                val_labels.extend(batch_labels.cpu().numpy())
        
        train_precision, train_recall, train_f1, _ = precision_recall_fscore_support(train_labels, train_preds, average='binary')
        train_accuracy = accuracy_score(train_labels, train_preds)
        
        val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(val_labels, val_preds, average='macro', zero_division=1)
        val_accuracy = accuracy_score(val_labels, val_preds)
        
        scheduler.step(val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.4f}, F1: {train_f1:.4f}')
        print(f'Val Loss: {val_loss/len(val_loader):.4f}, Accuracy: {val_accuracy:.4f}, F1: {val_f1:.4f}')
        print(f'Val Precision: {val_precision:.4f}, Recall: {val_recall:.4f}\n')
        
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save({
                'model_state_dict': model.state_dict(),
                'preprocessor': preprocessor,
                'config': config
            }, 'best_model.pt')
            patience_counter = 0
        else:
            patience_counter += 1
            
        if patience_counter >= patience:

            print(f'Early stopping triggered after epoch {epoch+1}')
            break

# -----------------------------
# 5. Configuration and Data Loading
# -----------------------------
config = {
    'max_vocab_size': 15000,
    'max_seq_length': 128,
    'embed_dim': 300,  # Must match the dimension of the pretrained embeddings
    'hidden_dim': 256,
    'batch_size': 16,
    'learning_rate': 0.001,
    'num_epochs': 20,
    'num_lstm_layers': 2,
    'pretrained_embedding_path': '/Users/User/CSProjects/CSC392_AI_agent/emphatic-AI-Winter2025/glove.6B/glove.6B.300d.txt'  # Update this path as needed
}  

# Load your datasets (assumed to be in TSV format)
train_df = pd.read_csv('train.tsv', sep='\t')
test_df = pd.read_csv('test.tsv', sep='\t')

# Map your string labels to numerical values
label_mapping = {'NOCUOUS': 0, 'INNOCUOUS': 1}
train_df['Detected as'] = train_df['Detected as'].map(label_mapping)
test_df['Detected as'] = test_df['Detected as'].map(label_mapping)

# -----------------------------
# 6. Preprocessing
# -----------------------------
preprocessor = TextPreprocessor(max_vocab_size=config['max_vocab_size'], 
                                max_seq_length=config['max_seq_length'])
preprocessor.fit(train_df['Sentence'])

X_train = preprocessor.transform(train_df['Sentence'])
X_val = preprocessor.transform(test_df['Sentence'])
y_train = torch.tensor(train_df['Detected as'].values)
y_val = torch.tensor(test_df['Detected as'].values)

train_dataset = TextDataset(X_train, y_train)
val_dataset = TextDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'])

x_np = X_train.numpy()
y_np = y_train.numpy()

# ----------------------------
# 7. 10-Fold Cross Validation
# ----------------------------

num_folds = 10
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

fold_results = []

for fold, (train_idx, val_idx) in enumerate(skf.split(x_np, y_np)): 
    print(f"\n------- Fold {fold+1} / {num_folds} --------- ")

    # -----------------------------
    # 1. Split Data
    # -----------------------------
    X_train_fold = torch.tensor(x_np[train_idx])
    y_train_fold = torch.tensor(y_np[train_idx])
    X_val_fold = torch.tensor(x_np[val_idx])
    y_val_fold = torch.tensor(y_np[val_idx])

    train_dataset = TextDataset(X_train_fold, y_train_fold)
    val_dataset = TextDataset(X_val_fold, y_val_fold)

    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'])

    # -----------------------------
    # 2. Initialize Model, Optimizer, and Criterion Inside the Loop
    # -----------------------------
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = ImprovedTextClassifier(
        vocab_size=len(preprocessor.word2idx),
        embed_dim=config['embed_dim'],
        hidden_dim=config['hidden_dim'],
        num_classes=2,
        num_layers=config['num_lstm_layers']
    ).to(device)

    # Load pretrained embeddings and replace the embedding layer weights
    pretrained_weights = load_pretrained_embeddings(config['pretrained_embedding_path'], 
                                                    preprocessor.word2idx, 
                                                    config['embed_dim'])
    model.embedding = nn.Embedding.from_pretrained(pretrained_weights, freeze=False, padding_idx=0)

    # Compute class weights to handle class imbalance
    small_factor = 1e-4
    class_counts = torch.bincount(y_train_fold) + small_factor  # Avoid division by zero
    class_weights = 1.0 / class_counts.float()
    class_weights = class_weights / class_weights.sum()
    class_weights = class_weights.to(device)

    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=0.01)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

    # -----------------------------
    # 3. Train the Model
    # -----------------------------
    train_model(model, train_loader, val_loader, criterion, optimizer, scheduler,
                config['num_epochs'], device, preprocessor, config)

    # Load the best model for this fold
    with torch.serialization.safe_globals([TextPreprocessor]):
        checkpoint = torch.load('best_model.pt', weights_only=False)
    
    model.load_state_dict(checkpoint['model_state_dict'])

    # -----------------------------
    # 4. Evaluate the Model
    # -----------------------------
    model.eval()
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for batch_texts, batch_labels in val_loader:
            batch_texts, batch_labels = batch_texts.to(device), batch_labels.to(device)
            outputs = model(batch_texts)
            predictions = torch.argmax(outputs, dim=1)
            val_preds.extend(predictions.cpu().numpy())
            val_labels.extend(batch_labels.cpu().numpy())

    # Compute metrics
    val_precision, val_recall, val_f1, _ = precision_recall_fscore_support(val_labels, val_preds, average='binary')
    val_accuracy = accuracy_score(val_labels, val_preds)

    print(f"Fold {fold+1} Metrics:")
    print(f"Accuracy: {val_accuracy:.4f}, F1-score: {val_f1:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}\n")

    fold_results.append((val_accuracy, val_f1, val_precision, val_recall))

# -----------------------------
# 5. Display Overall Results
# -----------------------------
avg_accuracy = sum([x[0] for x in fold_results]) / num_folds
avg_f1 = sum([x[1] for x in fold_results]) / num_folds
avg_precision = sum([x[2] for x in fold_results]) / num_folds
avg_recall = sum([x[3] for x in fold_results]) / num_folds

print("\n========== Final Cross-Validation Results ==========")
print(f"Avg Accuracy: {avg_accuracy:.4f}, Avg F1-score: {avg_f1:.4f}, Avg Precision: {avg_precision:.4f}, Avg Recall: {avg_recall:.4f}")


------- Fold 1 / 10 --------- 
Loading pretrained embeddings...
Found 781 pretrained embeddings out of 999 words


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20:
Train Loss: 0.6935, Accuracy: 0.6160, F1: 0.6000
Val Loss: 0.6889, Accuracy: 0.5714, F1: 0.3636
Val Precision: 0.7857, Recall: 0.5000

Epoch 2/20:
Train Loss: 0.6790, Accuracy: 0.5760, F1: 0.5954
Val Loss: 0.6938, Accuracy: 0.4286, F1: 0.4167
Val Precision: 0.4500, Recall: 0.4583

Epoch 3/20:
Train Loss: 0.4158, Accuracy: 0.8000, F1: 0.8000
Val Loss: 0.6845, Accuracy: 0.5714, F1: 0.3636
Val Precision: 0.7857, Recall: 0.5000

Epoch 4/20:
Train Loss: 0.3368, Accuracy: 0.8880, F1: 0.8852
Val Loss: 0.6877, Accuracy: 0.5714, F1: 0.3636
Val Precision: 0.7857, Recall: 0.5000

Epoch 5/20:
Train Loss: 0.4264, Accuracy: 0.8320, F1: 0.8346
Val Loss: 0.6892, Accuracy: 0.5714, F1: 0.4750
Val Precision: 0.5417, Recall: 0.5208

Epoch 6/20:
Train Loss: 0.2371, Accuracy: 0.9120, F1: 0.9134
Val Loss: 0.7318, Accuracy: 0.5714, F1: 0.5333
Val Precision: 0.5500, Recall: 0.5417

Epoch 7/20:
Train Loss: 0.1230, Accuracy: 0.9760, F1: 0.9752
Val Loss: 0.8211, Accuracy: 0.6429, F1: 0.6410
Val Precis

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20:
Train Loss: 0.7580, Accuracy: 0.5280, F1: 0.5693
Val Loss: 0.6969, Accuracy: 0.4286, F1: 0.3000
Val Precision: 0.7143, Recall: 0.5000

Epoch 2/20:
Train Loss: 0.6284, Accuracy: 0.6160, F1: 0.6190
Val Loss: 0.6935, Accuracy: 0.5000, F1: 0.4759
Val Precision: 0.5606, Recall: 0.5417

Epoch 3/20:
Train Loss: 0.5490, Accuracy: 0.7360, F1: 0.7402
Val Loss: 0.6916, Accuracy: 0.7143, F1: 0.6889
Val Precision: 0.7250, Recall: 0.6875

Epoch 4/20:
Train Loss: 0.3939, Accuracy: 0.8160, F1: 0.8067
Val Loss: 0.6735, Accuracy: 0.7143, F1: 0.6889
Val Precision: 0.7250, Recall: 0.6875

Epoch 5/20:
Train Loss: 0.3238, Accuracy: 0.8560, F1: 0.8500
Val Loss: 0.6819, Accuracy: 0.6429, F1: 0.6410
Val Precision: 0.6778, Recall: 0.6667

Epoch 6/20:
Train Loss: 0.3326, Accuracy: 0.8880, F1: 0.8871
Val Loss: 0.7576, Accuracy: 0.6429, F1: 0.6410
Val Precision: 0.6778, Recall: 0.6667

Epoch 7/20:
Train Loss: 0.2153, Accuracy: 0.9280, F1: 0.9280
Val Loss: 0.8399, Accuracy: 0.5714, F1: 0.3636
Val Precis

...

Epoch 1/20:
Train Loss: 0.7342, Accuracy: 0.5760, F1: 0.5546
Val Loss: 0.6923, Accuracy: 0.5000, F1: 0.3333
Val Precision: 0.7500, Recall: 0.5000

Epoch 2/20:
Train Loss: 0.5887, Accuracy: 0.6720, F1: 0.6555
Val Loss: 0.6944, Accuracy: 0.5000, F1: 0.3333
Val Precision: 0.7500, Recall: 0.5000

Epoch 3/20:
Train Loss: 0.5088, Accuracy: 0.7680, F1: 0.7603
Val Loss: 0.7001, Accuracy: 0.5714, F1: 0.5625
Val Precision: 0.5778, Recall: 0.5714

Epoch 4/20:
Train Loss: 0.3849, Accuracy: 0.8640, F1: 0.8661
Val Loss: 0.6617, Accuracy: 0.4286, F1: 0.3000
Val Precision: 0.2308, Recall: 0.4286

Epoch 5/20:
Train Loss: 0.3017, Accuracy: 0.8800, F1: 0.8780
Val Loss: 0.7202, Accuracy: 0.7857, F1: 0.7754
Val Precision: 0.8500, Recall: 0.7857

Epoch 6/20:
Train Loss: 0.2245, Accuracy: 0.9040, F1: 0.9000
Val Loss: 0.8900, Accuracy: 0.7857, F1: 0.7846
Val Precision: 0.7917, Recall: 0.7857

Epoch 7/20:
Train Loss: 0.1350, Accuracy: 0.9680, F1: 0.9667
Val Loss: 1.1275, Accuracy: 0.4286, F1: 0.3000
Val Precis

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20:
Train Loss: 0.7644, Accuracy: 0.5040, F1: 0.5921
Val Loss: 0.6890, Accuracy: 0.5000, F1: 0.3333
Val Precision: 0.7500, Recall: 0.5000

Epoch 2/20:
Train Loss: 0.6415, Accuracy: 0.6320, F1: 0.6515
Val Loss: 0.6894, Accuracy: 0.5714, F1: 0.4750
Val Precision: 0.7692, Recall: 0.5714

Epoch 3/20:
Train Loss: 0.4477, Accuracy: 0.8320, F1: 0.8205
Val Loss: 0.6903, Accuracy: 0.5714, F1: 0.5625
Val Precision: 0.5778, Recall: 0.5714

Epoch 4/20:
Train Loss: 0.4553, Accuracy: 0.7680, F1: 0.7521
Val Loss: 0.7016, Accuracy: 0.5000, F1: 0.3333
Val Precision: 0.7500, Recall: 0.5000

Epoch 5/20:
Train Loss: 0.2312, Accuracy: 0.9280, F1: 0.9231
Val Loss: 0.7439, Accuracy: 0.4286, F1: 0.3000
Val Precision: 0.2308, Recall: 0.4286

Epoch 6/20:
Train Loss: 0.1689, Accuracy: 0.9600, F1: 0.9573
Val Loss: 0.8167, Accuracy: 0.4286, F1: 0.3000
Val Precision: 0.2308, Recall: 0.4286

Epoch 7/20:
Train Loss: 0.2467, Accuracy: 0.9040, F1: 0.8966
Val Loss: 0.9061, Accuracy: 0.5714, F1: 0.5625
Val Precis

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20:
Train Loss: 0.7525, Accuracy: 0.5794, F1: 0.6074
Val Loss: 0.6952, Accuracy: 0.4615, F1: 0.3158
Val Precision: 0.7308, Recall: 0.5000

Epoch 2/20:
Train Loss: 0.5937, Accuracy: 0.6984, F1: 0.7206
Val Loss: 0.6966, Accuracy: 0.4615, F1: 0.3158
Val Precision: 0.7308, Recall: 0.5000

Epoch 3/20:
Train Loss: 0.5428, Accuracy: 0.7222, F1: 0.7287
Val Loss: 0.6839, Accuracy: 0.5385, F1: 0.3500
Val Precision: 0.7692, Recall: 0.5000

Epoch 4/20:
Train Loss: 0.3487, Accuracy: 0.8492, F1: 0.8455
Val Loss: 0.7021, Accuracy: 0.5385, F1: 0.3500
Val Precision: 0.7692, Recall: 0.5000

Epoch 5/20:
Train Loss: 0.2916, Accuracy: 0.8810, F1: 0.8739
Val Loss: 0.8636, Accuracy: 0.5385, F1: 0.3500
Val Precision: 0.7692, Recall: 0.5000

Epoch 6/20:
Train Loss: 0.2169, Accuracy: 0.9206, F1: 0.9138
Val Loss: 0.8874, Accuracy: 0.3077, F1: 0.2909
Val Precision: 0.2875, Recall: 0.2976

Epoch 7/20:
Train Loss: 0.0790, Accuracy: 0.9921, F1: 0.9916
Val Loss: 1.0984, Accuracy: 0.3077, F1: 0.3077
Val Precis